In [13]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from __future__ import print_function
import math

data_file = 'final_count_all.txt'
testData_percent = 0.2

# RF_Parameters
num_trees = 300
max_depth = 4
max_bins = 400
feature_subset_mode = 'log2'
cat_fea_info = {}
cat_fea_info[0] = 266
#cat_fea_info[1] = 24
cat_fea_info[2] = 2

#Initial Spark Environment
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName("RandomForestRegressor").getOrCreate()

# Load and parse the data file into an RDD of LabeledPoint.
data = MLUtils.loadLibSVMFile(sc, data_file)
# Split the data into trainData and testData
(trainData, testData) = data.randomSplit([1-testData_percent, testData_percent])

# Train RandomForest Regressor
RF = RandomForest.trainRegressor(trainData, categoricalFeaturesInfo=cat_fea_info,
                                    numTrees=num_trees, featureSubsetStrategy=feature_subset_mode,
                                    impurity='variance', maxDepth=max_depth, maxBins=max_bins, seed=None)


# Test model and compute Root Mean Square Error
predictions = RF.predict(testData.map(lambda row: row.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testMSE = labelsAndPredictions.map(lambda lp: (lp[0] - lp[1]) * (lp[0] - lp[1])).sum() / float(testData.count())
testRMSE = math.sqrt(testMSE)

print('Test Root Mean Squared Error on ' + data_file + ' = ' + str(testRMSE))
#print('Learned regression forest model:')
#print(model.toDebugString())

#RF.save(sc, "./RF_tip_model")

# Sample predictions to dataframe and show them
#predict_sample = predictions.sample(False, 10/predictions.count())
#predict_df = predict_sample.map(lambda x: (x, )).toDF()
predict_df = predictions.map(lambda x: (x, )).toDF()
predict_df.cache()
predict_df.show(5, False)

# Save and load model
RF.save(sc, "./RF_count_model")
#RF_load = RandomForestModel.load(sc, "./RF_tip_model")
#RF_load.predict([100, 10, 0])


Test Root Mean Squared Error on final_count_all.txt = 8.790447477369113
+------------------+
|_1                |
+------------------+
|1.9302946875972995|
|3.5442637703538873|
|2.1790349551061596|
|8.628741580762169 |
|1.6803249078859674|
+------------------+
only showing top 5 rows



In [14]:
feature_input = [162, 10, 0]

RF_count = RandomForestModel.load(sc, "./RF_count_model")
print('Count:')
print(RF_count.predict(feature_input))

RF_fare = RandomForestModel.load(sc, "./RF_fare_model")
print('Fare:')
print(RF_fare.predict(feature_input))

RF_tip = RandomForestModel.load(sc, "./RF_tip_model")
print('Tips:')
print(RF_tip.predict(feature_input))

Count:
6.046717153094052
Fare:
11.066386195072631
Tips:
0.9744953231648962


Fare:
16.995850288784332
